Callback est une fonction de rappel, le serveur l'appele après avoir reçu une demande, il va expédier la commande au module approprié. Il manque l'interface "index.html".

In [ ]:
# %load AjaxHandler.py
#!/usr/bin/env python3

import http.server  
      
callback = None  
      
class AjaxHandler(http.server.BaseHTTPRequestHandler):  
    def do_GET(self):  
        contenttype = callback(self.path)  
        self.send_response(200)  
        self.send_header("Content-type", contenttype[1])  
        self.end_headers()  
        self.wfile.write(contenttype[0])

In [6]:
import AjaxHandler
import http.server 

PORT = 8080  

In [7]:
def AjaxHandlerCallbackFunc(path):  
    content = b"None"  
    contenttype = "text/plain"  
    pathlist = path.split('?', 1)  
    if(pathlist[0] == "/"):  
        if(len(pathlist) == 1):  
            f = open("index.html", "rb")  
            content = f.read()  
            contenttype = "text/html"  
            f.close()  
        else:  
            args = pathlist[1].split('&')  
            cmd = args[0].split('=')  
            if cmd[0] == "cmd":  
                params = {}  
                if len(args) > 1:  
                    for arg in args[1:]:  
                        param = arg.split('=')  
                        val = param[1]  
                        try:  
                            val = int(val)  
                        except ValueError:  
                            try:  
                                val = float(val)  
                            except ValueError:  
                                pass  
                        params[param[0]] = val  
                content = DispatchCmd(cmd[1], params)  
    elif(pathlist[0] == "/favicon.ico"):  
        f = open("test.png", "rb")  
        content = f.read()  
        contenttype = "image/png"  
        f.close()  
    return (content, contenttype) 
      
def DispatchCmd(cmd, args):  
    return str({cmd:args}).encode()

In [8]:
if __name__ == "__main__":  
    try:  
        AjaxHandler.callback = AjaxHandlerCallbackFunc  
        server = http.server.HTTPServer(("", PORT), AjaxHandler.AjaxHandler)  
        print("HTTP server is starting at port "+repr(PORT)+'...')  
        print("Press Ctrl+C to quit")  
        server.serve_forever()  
    except KeyboardInterrupt:  
        print("Shutting down server")  
        server.socket.close()  

HTTP server is starting at port 8080...
Press Ctrl+C to quit
Shutting down server
